In [ ]:
import xarray as xr
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import KBinsDiscretizer
from tqdm import tqdm  # Para mostrar una barra de progreso

# Ruta de entrada en el clúster
file_path = "/storage/ggarcia_g/imandt/Land_and_Ocean_EqualArea.nc"

# Abrir el dataset
data = xr.open_dataset(file_path)
latitudes = data['latitude'].values
longitudes = data['longitude'].values
temperature = data['temperature'].values

times = data['time'].values
lat_grid = np.linspace(latitudes.min(), latitudes.max(), 50)  # Reducido a 50 para mejorar el rendimiento
lon_grid = np.linspace(longitudes.min(), longitudes.max(), 50)
lon_grid_mesh, lat_grid_mesh = np.meshgrid(lon_grid, lat_grid)

# Definir las décadas basadas en la variable de tiempo (asumiendo que el tiempo está en años)
decades = np.floor(times / 10) * 10

# Inicializar un diccionario para almacenar las matrices de información mutua por década
mutual_info_matrices_by_decade = {}

# Procesar datos por década
unique_decades = np.unique(decades)

# Crear un directorio para guardar los datos
save_dir = '/storage/ggarcia_g/imandt/decade_mutual_info'
os.makedirs(save_dir, exist_ok=True)

# Inicializar el discretizador para la información mutua
# Usaremos 10 bins; ajusta según tus necesidades
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')

for decade in unique_decades:
    print(f"Procesando década: {int(decade)}")
    # Extraer datos para la década actual
    decade_mask = (decades == decade)
    temperature_decade = temperature[decade_mask, :]
    
    # Interpolar los datos en una malla regular para todos los puntos temporales en la década
    num_time_points_decade = temperature_decade.shape[0]
    temperature_grid_over_time_decade = np.empty((num_time_points_decade, 50, 50))
    
    for t in range(num_time_points_decade):
        temperature_grid_over_time_decade[t] = griddata(
            (latitudes, longitudes), 
            temperature_decade[t], 
            (lat_grid_mesh, lon_grid_mesh), 
            method='linear'
        )
    
    # Manejar posibles NaNs después de la interpolación
    temperature_grid_over_time_decade = np.nan_to_num(temperature_grid_over_time_decade, nan=np.nanmean(temperature_grid_over_time_decade))
    
    # Reshape de los datos para el cálculo de la información mutua
    reshaped_temperature_data_decade = temperature_grid_over_time_decade.reshape(num_time_points_decade, -1)
    
    # Discretizar los datos
    reshaped_temperature_data_discrete = discretizer.fit_transform(reshaped_temperature_data_decade)
    reshaped_temperature_data_discrete = reshaped_temperature_data_discrete.astype(int)
    
    num_grid_points = reshaped_temperature_data_discrete.shape[1]
    
    # Inicializar una matriz para la información mutua
    mutual_info_matrix_decade = np.empty((num_grid_points, num_grid_points))
    
    # Calcular la información mutua para cada par de puntos de la malla
    # Usamos tqdm para mostrar una barra de progreso
    for i in tqdm(range(num_grid_points), desc=f"Calculando MI para década {int(decade)}"):
        for j in range(num_grid_points):
            if i <= j:  # Aprovechar la simetría para reducir cálculos
                mi = mutual_info_score(
                    reshaped_temperature_data_discrete[:, i],
                    reshaped_temperature_data_discrete[:, j]
                )
                mutual_info_matrix_decade[i, j] = mi
                mutual_info_matrix_decade[j, i] = mi  # Simetría
            else:
                continue  # Ya calculado
    
    # Almacenar la matriz de información mutua en el diccionario
    mutual_info_matrices_by_decade[int(decade)] = mutual_info_matrix_decade
    
    # Guardar la matriz de información mutua para esta década
    np.save(os.path.join(save_dir, f'mutual_info_matrix_decade_{int(decade)}.npy'), mutual_info_matrix_decade)

# Confirmación de las décadas guardadas
saved_decades = list(mutual_info_matrices_by_decade.keys())

# Guardar la lista de décadas
np.save(os.path.join(save_dir, 'saved_decades.npy'), np.array(saved_decades))

print("Cálculo de información mutua completado y guardado exitosamente.")
